In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import re
import plotly.express as px
import plotly.graph_objects as go
import chart_studio.plotly
chart_studio.tools.set_credentials_file(username='tom.hd', api_key='ofaW3SZfg0TMqQ6oiVVN')
chart_studio.tools.set_config_file(world_readable=True, sharing='public')
import chart_studio.plotly as py

In [ ]:
df_energy = pd.read_csv('data/Energy_Transition.csv')
df_energy.head()

In [ ]:
df_energy = df_energy.drop(['ObjectId', 'ISO3', 'Unit','Source', 'CTS_Name', 'CTS_Code', 'CTS_Full_Descriptor'], axis=1)
df_energy.rename({'Energy_Type': 'Energy Type'}, axis=1, inplace=True)
df_energy['Technology'].replace({'Hydropower (excl. Pumped Storage)': 'Hydropower', "Other non-renewable energy": "Other non-renewable"}, inplace=True)
df_energy['Country'].replace({'China, P.R.: Mainland': 'China', 'Korea, Rep. of': 'Korea', 'Russian Federation': 'Russia' }, inplace=True)

# regular expressions to format the years column names and units
for col in df_energy.columns:
    if re.match(r'F\d{4}', col):
        new_col = re.sub(r'F(\d{4})', r'\1', col)
        df_energy.rename(columns={col: new_col}, inplace=True)

df_energy.head()

In [ ]:
# checking for duplicates
print(df_energy.shape)
df_energy = df_energy.drop_duplicates()
print(df_energy.shape)

In [ ]:
# checking for duplicates in the subset of columns
df_energy.head()
duplicated = df_energy.duplicated(subset=['Country', 'Indicator', 'Technology', 'Energy Type'], keep=False)
df_energy[duplicated]

In [ ]:
# clustering minor energy types in renewable and non-renewable
df_energy['Technology'].replace({"Pumped storage": "Other non-renewable", "Marine energy": "Other renewable", "Geothermal energy": "Other renewable"}, inplace=True)

In [ ]:
df_gen = df_energy[df_energy['Indicator'] == 'Electricity Generation']
df_cap = df_energy[df_energy['Indicator'] == 'Electricity Installed Capacity']
df_world_gen = df_energy[(df_energy['Country'] == 'World') & (df_energy['Indicator'] == 'Electricity Generation')]
df_world_cap = df_energy[(df_energy['Country'] == 'World') & (df_energy['Indicator'] == 'Electricity Installed Capacity')]

# setting common colors
color_discrete_map={
                "Fossil fuels": px.colors.qualitative.Prism[10],
                "Hydropower": px.colors.qualitative.Prism[1],
                "Nuclear": px.colors.qualitative.Prism[7],
                "Wind energy": px.colors.qualitative.Prism[2],
                "Solar energy": px.colors.qualitative.Prism[5],
                "Other renewable": px.colors.qualitative.Prism[3],
                "Bioenergy": px.colors.qualitative.Prism[4],
                "Other non-renewable": px.colors.qualitative.Prism[0],
                }

In [ ]:
# checking for missing data
missing_percentages = df_world_gen.isna().mean() * 100

print(missing_percentages)

In [ ]:
# checking for missing data
missing_percentages = df_world_cap.isna().mean() * 100

print(missing_percentages)

In [ ]:
# Renewable vs. Non-renewable energy generation
df_world_gtype = df_world_gen.groupby(['Energy Type']).sum()
df_world_gtype.reset_index(inplace=True)

df_world_gtype = pd.melt(df_world_gtype, id_vars=['Energy Type'], var_name='Year', value_name='Energy generation (GWH)')

#Renweable vs. Non-renewable energy capacity
df_world_ctype = df_world_cap.groupby(['Energy Type']).sum()
df_world_ctype.reset_index(inplace=True)

df_world_ctype = pd.melt(df_world_ctype, id_vars=['Energy Type'], var_name='Year', value_name='Energy capacity (MW)')

In [ ]:
# calculating the percentage of renewable energy generation
numbers = df_world_gtype[df_world_gtype['Year'] == '2020']
total = numbers['Energy generation (GWH)'].sum()

print(numbers[numbers['Energy Type'] == 'Total Renewable']['Energy generation (GWH)'].values[0] / total * 100)
print(numbers[numbers['Energy Type'] == 'Total Non-Renewable']['Energy generation (GWH)'].values[0] / total * 100)

In [ ]:
# formatting the data for the stacked bar chart
df_wide = df_world_gtype.pivot(index='Year', columns='Energy Type', values='Energy generation (GWH)')
df_wide.reset_index(inplace=True)
df_wide.head()

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(
        x= df_wide['Year'], y = df_wide['Total Renewable'], 
        name = 'Total Renewable',
        mode = 'lines',
        line = dict(color = px.colors.qualitative.Prism[3]),
        fill = 'tozeroy'))
fig.add_trace(go.Scatter(
        x= df_wide['Year'], y = df_wide['Total Non-Renewable'],
        name = 'Total Non-Renewable',
        line=dict(color= px.colors.qualitative.Prism[7]),
        mode = 'lines',
        fill = 'tonexty'))
fig.update_layout(
        title = "Renewable vs. Non-renewable energy generation", 
        width = 800, height = 600,
        #legend_title_text = 'Energy Type',
        )
fig.update_xaxes(
     title_text = 'Year',
     tickmode = 'linear',
     dtick = 2)
fig.update_yaxes(
     title_text = "Energy generation (GWH)")

fig.show()
py.plot(fig, filename = 'Renewable vs Non-Renewable generation', auto_open=False)

In [ ]:
# grouping the wold energy generaton by technology
world_tech_gen = df_world_gen.groupby(['Technology']).sum()
world_tech_gen.reset_index(inplace=True)
world_tech_gen = pd.melt(world_tech_gen, id_vars=['Technology'], var_name='Year', value_name='Energy generation (GWH)')
world_tech_gen.head()

In [ ]:
fig = px.line(world_tech_gen, 
              x='Year', 
              y='Energy generation (GWH)',  
              title='Global energy generation by technology',
              color='Technology',
              color_discrete_map=color_discrete_map,
              category_orders={'Technology': world_tech_gen.sort_values('Energy generation (GWH)', ascending=False)['Technology']},
              height=600,width=800)
fig.update_layout(
    xaxis = dict(
        tickmode = 'linear',
        dtick = 2
    )
)
fig.show()
py.plot(fig, filename = 'Global energy generation by technology', auto_open=False)

In [ ]:
# grouping the wold energy capacity by technology
world_tech_cap = df_world_cap.groupby(['Technology']).sum()
world_tech_cap.reset_index(inplace=True)
world_tech_cap = pd.melt(world_tech_cap, id_vars=['Technology'], var_name='Year', value_name='Energy capacity (MW)')

In [ ]:
fig = px.line(world_tech_cap, 
              x='Year', 
              y='Energy capacity (MW)',  
              title='Global energy capacity by technology',
              color='Technology',
              color_discrete_map=color_discrete_map,
              category_orders={'Technology': world_tech_cap.sort_values('Energy capacity (MW)', ascending=False)['Technology']},
              height=600,width=800)
fig.update_layout(
    xaxis = dict(
        tickmode = 'linear',
        dtick = 2
    )
)
fig.show()
py.plot(fig, filename = 'Global capacity generation by technology', auto_open=False)

In [ ]:
# taking the most recent data as no values are missing
world_tech_cap20 = world_tech_cap[world_tech_cap['Year'] == '2020']

fig = px.bar(world_tech_cap20, 
              x='Energy capacity (MW)', 
              y='Technology',
              title='Global energy capacity in 2020',
              orientation='h',
              height=600,width=800,
              color='Technology',
              color_discrete_map=color_discrete_map,
              category_orders={'Technology': world_tech_cap20.sort_values('Energy capacity (MW)', ascending=False)['Technology']})

fig.show()
py.plot(fig, filename = 'Global energy capacity in 2020', auto_open=False)

In [ ]:
# filtering for countries
df_countries = df_energy.dropna(subset=['ISO2'])
df_countries = df_countries[df_countries['Indicator'] == 'Electricity Generation']

# filtering for the top 10 countries by generation in 2020
country_gen = df_countries.groupby('Country').sum()['2020']
country_gen = country_gen.sort_values(ascending=False)
country_gen = country_gen.reset_index()
top_10_gen = country_gen.iloc[:10]

# creating dataframe with energy generation by country and technology
df_top_10 = df_countries[df_countries['Country'].isin(top_10_gen['Country'].tolist())]

# checking for missing data
missing_percentages = df_top_10.isna().mean() * 100
print(missing_percentages)

In [ ]:
df_top_10 = df_top_10.groupby(['Country', 'Technology']).sum()['2020']
df_top_10 = df_top_10.reset_index()
df_top_10.sort_values('2020', ascending=False, inplace=True)
df_top_10.head()

In [ ]:
# dataframe to sort the the bar chart by total generation descending
top10_sum = df_top_10.groupby('Country').sum()
top10_sum = top10_sum.reset_index()
top10_sum.sort_values('2020', ascending=False, inplace=True)

In [ ]:
fig = px.bar(df_top_10, 
              x='Country', 
              y='2020',
              title='Top 10 energy generators in 2020',
              orientation='v',
              barmode='stack',
              height=600,width=900,
              color='Technology',
              color_discrete_map= color_discrete_map,
              category_orders={'Country': top10_sum['Country'].tolist()})
              
fig.update_yaxes(
        title_text = "Energy generation (GWH)"
)          
fig.show()
py.plot(fig, filename = 'Top 10 energy generators', auto_open=False)

In [ ]:
# grouping energy generation by country and technology
df_gtype = df_gen.groupby(['Country', 'Technology']).sum()
df_gtype.reset_index(inplace=True)
df_gtype = pd.melt(df_gtype, id_vars=['Country', 'Technology'], var_name='Year', value_name='Energy generation (GWH)')
df_gtype.head()

In [ ]:
# grouping energy capacity by country and technology
df_ctype = df_cap.groupby(['Country', 'Technology']).sum()
df_ctype.reset_index(inplace=True)
df_ctype = pd.melt(df_ctype, id_vars=['Country', 'Technology'], var_name='Year', value_name='Energy capacity (MW)')

In [ ]:
fig = px.line(df_ctype[df_ctype['Country'] == 'China'], 
              x='Year', 
              y='Energy capacity (MW)',  
              title='China\'s energy mix',
              color='Technology',
              color_discrete_map=color_discrete_map,
              height=500,width=700,
              category_orders={'Technology': df_ctype[df_ctype['Country'] == 'China'].sort_values('Energy capacity (MW)', ascending=False)['Technology']}
              )

fig.update_layout(
    xaxis = dict(
        tickmode = 'linear',
        dtick = 2
    )
)
fig.show()
py.plot(fig, filename = 'China Energy Mix', auto_open=False)

In [ ]:
fig = px.line(df_ctype[df_ctype['Country'] == 'United States'], 
              x='Year', 
              y='Energy capacity (MW)',  
              title='US energy mix',
              color='Technology',
              color_discrete_map=color_discrete_map,
              height=500,width=700,
              category_orders={'Technology': df_ctype[df_ctype['Country'] == 'Unites States'].sort_values('Energy capacity (MW)', ascending=False)['Technology']}
              )

fig.update_layout(
    xaxis = dict(
        tickmode = 'linear',
        dtick = 2
    )
)
fig.show()
py.plot(fig, filename = 'US Energy Mix', auto_open=False)

In [ ]:
fig = px.line(df_ctype[df_ctype['Country'] == 'India'], 
              x='Year', 
              y='Energy capacity (MW)',  
              title='India\'s energy mix',
              color='Technology',
              color_discrete_map=color_discrete_map,
              height=500,width=700,
              category_orders={'Technology': df_ctype[df_ctype['Country'] == 'India'].sort_values('Energy capacity (MW)', ascending=False)['Technology']}
              )

fig.update_layout(
    xaxis = dict(
        tickmode = 'linear',
        dtick = 2
    )
)
fig.show()
py.plot(fig, filename = 'India Energy Mix', auto_open=False)

In [ ]:
fig = px.line(df_ctype[df_ctype['Country'] == 'Russia'], 
              x='Year', 
              y='Energy capacity (MW)',  
              title='Russia\'s energy mix',
              color='Technology',
              color_discrete_map=color_discrete_map,
              height=500,width=700,
              category_orders={'Technology': df_ctype[df_ctype['Country'] == 'Russia'].sort_values('Energy capacity (MW)', ascending=False)['Technology']}
              )

fig.update_layout(
    xaxis = dict(
        tickmode = 'linear',
        dtick = 2
    )
)
fig.show()
py.plot(fig, filename = 'Russia Energy Mix', auto_open=False)

In [ ]:
# filtering for renewable energy capacity and grouping by technology
df_ren_cap = df_world_cap[df_world_cap['Energy Type'] == 'Total Renewable']
df_ren_cap = df_ren_cap.groupby(['Technology']).sum()
df_ren_cap.reset_index(inplace=True)
df_ren_cap = pd.melt(df_ren_cap, id_vars=['Technology'], var_name='Year', value_name='Energy capacity (MW)')
df_ren_cap.head()

In [ ]:
# formatting the data for the stacked bar chart
df_wide = df_ren_cap.pivot(index='Year', columns='Technology', values='Energy capacity (MW)')
df_wide.reset_index(inplace=True)

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(
        x= df_wide['Year'], y = df_wide['Other renewable'],
        name = 'Other renewable',
        mode = 'lines',
        line = dict(color = px.colors.qualitative.Prism[3]),
        fill = 'tonexty'))
fig.add_trace(go.Scatter(
        x= df_wide['Year'], y = df_wide['Bioenergy'],
        name = 'Bioenergy',
        mode = 'lines',
        line = dict(color = px.colors.qualitative.Prism[4]),
        fill = 'tonexty'))
fig.add_trace(go.Scatter(
        x= df_wide['Year'], y = df_wide['Solar energy'], 
        name = 'Solar energy',
        mode = 'lines',
        line = dict(color = px.colors.qualitative.Prism[5]),
        fill = 'tonexty'))
fig.add_trace(go.Scatter(
        x= df_wide['Year'], y = df_wide['Wind energy'], 
        name = 'Wind energy',
        mode = 'lines',
        line = dict(color = px.colors.qualitative.Prism[2]),
        fill = 'tonexty'))
fig.add_trace(go.Scatter(
        x= df_wide['Year'], y = df_wide['Hydropower'], 
        name = 'Hydropower',
        mode = 'lines',
        line = dict(color = px.colors.qualitative.Prism[1]),
        fill = 'tonexty'))
fig.update_layout(
        title = "Global renweable energy trend", 
        width = 800, height = 600)
fig.update_xaxes(
     title_text = 'Year',
     tickmode = 'linear',
     dtick = 2)
fig.update_yaxes(
     title_text = "Energy capacity (MW)")

fig.show()
py.plot(fig, filename = 'Renweable energy capacity trend', auto_open=False)

In [ ]:
# fossil fuel share by country
df_fossil = df_countries
df_sum = df_fossil.groupby(['Country'])['2020'].sum().reset_index()
df_sum.rename(columns={'2020': 'Total'}, inplace=True)

df_fossil = df_fossil[df_fossil['Technology'] == 'Fossil fuels']
cols_to_drop = [col for col in df_fossil.columns if col not in ['Country', '2020']]
df_fossil = df_fossil.drop(cols_to_drop, axis=1)
df_fossil.rename(columns={'2020': 'Fossil fuels'}, inplace=True)

df_share = pd.merge(df_fossil, df_sum, on='Country')
df_share['Share'] = df_share['Fossil fuels'] / df_share['Total'] * 100
df_share.drop(['Fossil fuels', 'Total'], axis=1, inplace=True)

df_share.head()

In [ ]:
fig = px.choropleth(df_share,   
            locations='Country',
            locationmode='country names',
            color='Share',
            title='Fossil fuel share across the world',
            color_continuous_scale="Reds",
            scope='world',
            fitbounds='locations',
            height=600,
            width=1000)
fig.update_layout(

    geo=dict(
        showframe=False,
        showcoastlines=False,
        projection_type='equirectangular'
        
    )
)

fig.show()
py.plot(fig, filename = 'Fossil fuel share across the world', auto_open=False)

In [ ]:
df_fs = pd.read_csv('data/fossil_fuel_support.csv')
df_fs.head()

In [ ]:
df_fs = df_fs[(df_fs['Support Mechanism'] == 'Total') & (df_fs['Beneficiary or Sector'] == 'Total')]
df_fs = df_fs.drop(['COUNTRY', 'MECHANISM', 'INDIC_OR_SEC', 'Beneficiary or Sector', 'PROD', 'MEASURE', 'YEAR', 'Flag Codes', 'Flags'], axis=1)
df_fs['Country'].replace({'China (People\'s Republic of)': 'China'}, inplace=True)

In [ ]:
# checking for duplicates
print(df_fs.shape)
df_fs = df_fs.drop_duplicates()
print(df_fs.shape)

In [ ]:
# checking for duplicated in the subset
df_fs.head()
duplicated = df_fs.duplicated(subset=['Country', 'Support Mechanism', 'Fuel Type', 'Unit', 'Year'], keep=False)
df_fs[duplicated]

In [ ]:
g20 = df_fs[df_fs['Country'] == 'G20 and EaP partner economies']
g20

In [ ]:
# checking for missing values
g20 = g20.replace(0, np.nan)
missing_percentages = g20.isna().mean() * 100
print(missing_percentages)

In [ ]:
g20_dollar = g20[(g20['Unit'] == 'US Dollars (nominal) [in millions]') & (g20['Fuel Type'] != 'Total')]
g20_dollar = g20_dollar.groupby(['Year', 'Fuel Type'])['Value'].sum()
g20_dollar = g20_dollar.reset_index()
g20_dollar.head()

In [ ]:
fig = px.bar(g20_dollar, 
              x='Year', 
              y='Value',  
              title='Fossil fuel support of G20',
              color='Fuel Type',
              height=600,width=800,
              category_orders={'Fuel Type': g20_dollar.sort_values('Value', ascending=False)['Fuel Type'].unique()},
              color_discrete_map={'Petroleum': px.colors.qualitative.Prism[6], 'End-use electricity': px.colors.qualitative.Prism[2], 'Natural gas': px.colors.qualitative.Prism[1], 'Coal': 'black'})
fig.update_layout(
    xaxis = dict(
        tickmode = 'linear',
        dtick = 1
    )
)

fig.update_yaxes(
        title_text = "US Dollars (nominal) [in millions]"
)     
fig.show()
py.plot(fig, filename = 'Fossil fuel support by country', auto_open=False)